In [1]:
import numpy as np
import pandas as pd
import taxcalc as tc
import microdf as mdf

In [2]:
run 'convert_asec_taxcalc.py'

In [3]:
run 'make_tax_units.py'

In [24]:
person = pd.read_csv('~/UBICenter/covid_ui/asec_2019_ipums.csv.gz')

## Preprocessing

In [25]:
# Set columns to lowercase and to 0 or null as appropriate.
prep_ipum(person)
# Add taxid and related fields.
tax_unit_id(person)
# Add other person-level columns in taxcalc form.
person = convert_asec_person_taxcalc(person)
# 99 is the missing code for wksunem1.
# Note: Missing codes for features used in taxcalc are recoded in
# convert_asec_taxcalc.py.
person.loc[person.wksunem1 == 99, 'wksunem1'] = 0

## Add UI to person records

In [26]:
FPUC_MAX_WEEKS = 17  # April to July.
FPUC2_MAX_WEEKS = 17 + 22  # Adding August to December.
FPUC_WEEKLY_BEN = 600
person['fpuc'] = FPUC_WEEKLY_BEN * np.minimum(person.wksunem1, FPUC_MAX_WEEKS)
person['fpuc2'] = FPUC_WEEKLY_BEN * np.minimum(person.wksunem1,
                                               FPUC2_MAX_WEEKS)

In [27]:
# Store original unemployment benefits.
person['e02300_orig'] = person.e02300

## Create tax units and calculate tax liability

In [28]:
# Create tax unit dataframe.
tu = create_tax_unit(person)
tu['RECID'] = tu.taxid
tu['tax'] = mdf.calc_df(records=tc.Records(tu), year=2018).tax.values

Simulate FPUC.

In [29]:
# Create tax unit dataframe.
person.e02300 = person.e02300_orig + person.fpuc
tu_fpuc = create_tax_unit(person)
tu['e02300_fpuc'] = tu_fpuc.e02300
tu_fpuc['RECID'] = tu_fpuc.taxid
tu['tax_fpuc'] = mdf.calc_df(records=tc.Records(tu_fpuc),
                             year=2018).tax.values
del tu_fpuc

Simulate extended FPUC.

In [30]:
# Create tax unit dataframe.
person.e02300 = person.e02300_orig + person.fpuc2
tu_fpuc2 = create_tax_unit(person)
tu['e02300_fpuc2'] = tu_fpuc2.e02300
tu_fpuc2['RECID'] = tu_fpuc2.taxid
tu['tax_fpuc2'] = mdf.calc_df(records=tc.Records(tu_fpuc2),
                             year=2018).tax.values
del tu_fpuc2

## Merge back to the person level

Have each person 

In [11]:
tu[['e02300', 'e02300_fpuc', 'tax', 'tax_fpuc']]

,e02300,e02300_fpuc,tax,tax_fpuc
0,0,0,4565.618219,4565.618219
1,0,0,0.000000,0.000000
2,0,0,2617.158353,2617.158353
3,0,0,0.000000,0.000000
4,0,0,0.000000,0.000000
...,...,...,...,...
86394,0,0,5261.231869,5261.231869
86395,0,0,5260.960374,5260.960374
86396,0,0,3585.158453,3585.158453
86397,0,0,21518.695306,21518.695306


In [12]:
(person.wksunem1 > 0).mean()

0.030771622589546976

In [15]:
(tu.e02300 > 0).mean()

0.019583559994907348

In [13]:
(tu.e02300_fpuc > 0).mean()

0.07105406312573062

In [19]:
tu.e02300_fpuc.sum() / tu.e02300.sum()

5.658077231808007

In [31]:
tu.tax_fpuc.sum() / tu.tax.sum()

1.0006684382444695

In [32]:
tu.tax_fpuc2.sum() / tu.tax.sum()

1.0009234493486514

In [14]:
tu.e02300_fpuc.describe()

count    86399.000000
mean       537.477193
std       2397.317471
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      54088.000000
Name: e02300_fpuc, dtype: float64